In [2]:
#importação das bibliotecas
import os
import pandas as pd
from pandas_gbq import to_gbq
import numpy as  np
import google.cloud.bigquery
from google.cloud import bigquery
from google.oauth2 import service_account
from binance.client import Client 

In [3]:
#Criação do objeto virtual para GCP 
credentials = service_account.Credentials.from_service_account_file("GBQ.json", 
                                                                    scopes=['https://www.googleapis.com/auth/cloud-platform',
                                                                            'https://www.googleapis.com/auth/drive'])


In [4]:
# Conexão com API da Binance
api_key = os.getenv("api_key_binance")
api_secret = os.getenv("api_secret_binance")

client = Client(api_key, api_secret)

In [5]:
#Função de tratamento dos dados recebidos

def tratamento_cripto(symbol):
    
    # extração dos dados históricos
    ar = np.array(client.get_historical_klines(symbol=symbol,
                                            interval = Client.KLINE_INTERVAL_1DAY, 
                                            start_str = "17, fev 2025"))

    #Criação do Dataframe
    df = pd.DataFrame(ar, dtype=float, columns = ("Open_Time", 
                                                "Open", 
                                                'High', 
                                                'Low', 
                                                'Closer', 
                                                'Volume', 
                                                'Close_Time',
                                                'Quote_asset_volume',
                                                'Number_of_Trades',
                                                'Take buy base asset',
                                                'Take buy quote asset',
                                                'Ignore'))

    # Definindo as colunas necessárias
    df = df[['Open_Time', 
            'Open', 
            'High', 
            'Low', 
            'Closer', 
            'Volume', 
            'Close_Time',
            'Quote_asset_volume',
            'Number_of_Trades']]

    #Tratamento de dados (date_time)

    df['Open_Time'] = pd.to_datetime(df['Open_Time'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
    df['Close_Time'] = pd.to_datetime(df['Close_Time'], unit = 'ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
    
    # Criação de novas colunas para métricas
    
    # Variação do preço (%)
    df['Variacao_Preco_%'] = (df['Closer'] - df['Open']) / df['Open'] * 100
    
    # Amplitude da variação
    df['Amplitude_Variacao'] = df['High'] - df['Low']

        
    return df

In [6]:
# Moedas selecionadas
btc = tratamento_cripto(symbol='BTCBRL')
eth = tratamento_cripto(symbol='ETHBRL')
sol = tratamento_cripto(symbol='SOLBRL')

In [7]:
# Enviar dados para o GBQ

to_gbq(btc, destination_table='BD_Binance_API.btc', project_id='etl-cgp', if_exists='replace', credentials=credentials)
to_gbq(eth, destination_table='BD_Binance_API.eth', project_id='etl-cgp', if_exists='replace', credentials=credentials)
to_gbq(sol, destination_table='BD_Binance_API.sol', project_id='etl-cgp', if_exists='replace', credentials=credentials)
